In [1]:
import sys
cpath = !pwd
sys.path.append(cpath[0][:-22])

In [2]:
import pandas as pd 
import numpy as np
from datasets.dataset import Dataset
from lionforests import LionForests
segment = Dataset()
import numpy as np
np.seterr(invalid='ignore')
import warnings
warnings.filterwarnings("ignore")
import time

In [3]:
X, y, feature_names, class_names = segment.load_segment()

In [6]:
from tqdm import tqdm
import time  #10 trees

results = []
iterr = 100
with tqdm(total=2*1*4*24) as pbar:
    for i in [1,5,7, 10]:
        for j in [10, 100, 500, 1000]:
            for o in ['sqrt','log2', 0.75, None]:
                parameters = [{
                    'max_depth': [i],
                    'max_features': [o],
                    'bootstrap': [False],
                    'min_samples_leaf' : [1],
                    'n_estimators': [j]
                }]
                lf = LionForests(None, False, None, feature_names, class_names)
                lf.fit(X, y, params=parameters)
                tlf = 0
                fr = 0
                pr = 0
                for inde in range(iterr):
                    t_a = time.time()
                    a, b, c, d, e, g = lf.explain(X[inde], False, None, None, False)
                    fr = fr + c - e
                    pr = pr + b - d
                    tlf = tlf + time.time() - t_a
                results.append([str(i),str(j),str(o),'NoRed','_','_', tlf/iterr, fr/iterr, pr/iterr])
                pbar.update(1)
                for k in ['1','2','3','12','13','23','123']:
                        if '1' in k and '2' in k:
                            for ara in ['apriori','fpgrowth']:
                                for cla in ['kmedoids','OPTICS','SC']:
                                    tlf = 0
                                    fr = 0
                                    pr = 0
                                    for inde in range(iterr):
                                        t_a = time.time()
                                        a, b, c, d, e, g = lf.explain(X[inde], True, ara, cla, method=k)
                                        fr = fr + c - e
                                        pr = pr + b - d
                                        tlf = tlf + time.time() - t_a
                                    results.append([str(i),str(j),str(o),k,ara,cla, tlf/iterr, fr/iterr, pr/iterr])
                                    pbar.update(1)
                        elif '1' in k:
                            for ara in ['apriori','fpgrowth']:
                                tlf = 0
                                fr = 0
                                pr = 0
                                for inde in range(iterr):
                                    t_a = time.time()
                                    a, b, c, d, e, g = lf.explain(X[inde], True, ara, None, method=k)
                                    fr = fr + c - e
                                    pr = pr + b - d
                                    tlf = tlf + time.time() - t_a
                                results.append([str(i),str(j),str(o),k,ara,'_', tlf/iterr, fr/iterr, pr/iterr])
                                pbar.update(1)
                        elif '2' in k:
                            for cla in ['kmedoids','OPTICS','SC']: 
                                tlf = 0
                                fr = 0
                                pr = 0
                                for inde in range(iterr):
                                    t_a = time.time()
                                    a, b, c, d, e, g = lf.explain(X[inde], True, None, cla, method=k)
                                    fr = fr + c - e
                                    pr = pr + b - d
                                    tlf = tlf + time.time() - t_a
                                results.append([str(i),str(j),str(o),k,'_',cla, tlf/iterr, fr/iterr, pr/iterr])
                                pbar.update(1)
                        else:
                                tlf = 0
                                fr = 0
                                pr = 0
                                for inde in range(iterr):
                                    t_a = time.time()
                                    a, b, c, d, e, g = lf.following_breadcrumbs(X[inde], True, None, None, method=k)
                                    fr = fr + c - e
                                    pr = pr + b - d
                                    tlf = tlf + time.time() - t_a
                                results.append([str(i),str(j),str(o),k,'_','_', tlf/iterr, fr/iterr, pr/iterr])
                                pbar.update(1)

100%|████████████████████████████████████████████████████████████████████████████| 192/192 [29:29:10<00:00, 552.86s/it]


In [7]:
for i in results:
    print(i)

['7', '1000', 'sqrt', 'NoRed', '_', '_', 0.8906031799316406, 0.0, 0.0]
['7', '1000', 'sqrt', '1', 'apriori', '_', 0.9058027696609497, 2.04, 215.05]
['7', '1000', 'sqrt', '1', 'fpgrowth', '_', 0.940398919582367, 2.04, 215.05]
['7', '1000', 'sqrt', '2', '_', 'kmedoids', 6.788333294391632, 0.92, 434.85]
['7', '1000', 'sqrt', '2', '_', 'OPTICS', 7.122288558483124, 0.12, 101.31]
['7', '1000', 'sqrt', '2', '_', 'SC', 8.942718441486358, 2.4, 437.34]
['7', '1000', 'sqrt', '3', '_', '_', 0.9637437772750854, 0.2, 448.43]
['7', '1000', 'sqrt', '12', 'apriori', 'kmedoids', 5.514538416862488, 2.82, 437.59]
['7', '1000', 'sqrt', '12', 'apriori', 'OPTICS', 5.551048257350922, 2.08, 224.64]
['7', '1000', 'sqrt', '12', 'apriori', 'SC', 7.288916394710541, 3.88, 441.44]
['7', '1000', 'sqrt', '12', 'fpgrowth', 'kmedoids', 5.217862722873687, 2.82, 437.59]
['7', '1000', 'sqrt', '12', 'fpgrowth', 'OPTICS', 5.272133109569549, 2.08, 224.64]
['7', '1000', 'sqrt', '12', 'fpgrowth', 'SC', 7.262777225971222, 3.88, 